In [1]:
!pip install nltk xgboost


Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score
import pickle
from typing import Dict
from xgboost import XGBRegressor




In [3]:
TARGETS_1 = ['Domínio da modalidade escrita formal_nota',                                                                                                  
 'Selecionar, relacionar, organizar e interpretar informações em defesa de um ponto de vista_nota',                                                 
 'Compreender a proposta e aplicar conceitos das várias áreas de conhecimento para desenvolver o texto dissertativo-argumentativo em prosa_nota',   
'Conhecimento dos mecanismos linguísticos necessários para a construção da argumentação_nota',                                                     
'Proposta de intervenção com respeito aos direitos humanos_nota']           

TARGETS_1_RENAME = {'Domínio da modalidade escrita formal_nota':'c1',                                                                                                  
 'Compreender a proposta e aplicar conceitos das várias áreas de conhecimento para desenvolver o texto dissertativo-argumentativo em prosa_nota':'c2',   
 'Selecionar, relacionar, organizar e interpretar informações em defesa de um ponto de vista_nota':'c3',                                                 
'Conhecimento dos mecanismos linguísticos necessários para a construção da argumentação_nota':'c4',                                                     
'Proposta de intervenção com respeito aos direitos humanos_nota':'c5'}           



TARGETS_3 = ['Conteúdo_nota',                                                                                                                               
'Estrutura do texto_nota',                                                                                                               
'Estrutura de ideias_nota',                                                                                                                 
'Vocabulário_nota',                                                                                                                       
'Gramática e ortografia_nota'] 

TARGETS_2 =  ['Adequação ao Tema_nota',                                                                                                                        
 'Adequação e Leitura Crítica da Coletânea_nota',                                                                                                
 'Adequação ao Gênero Textual_nota',                                                                                                               
 'Adequação à modalidade padrão da língua_nota',                                                                                                    
 'Coesão e Coerência_nota']

ID_VARS = ['index','tema','conjunto']

ALL_TARGETS = TARGETS_1  

EXCLUDE_COLS = ALL_TARGETS + ID_VARS
 
                                 
TF_IDF_MAX_FEATURES = [32,64]

### Juntar todos os jsons em um único arquivo

In [4]:
import glob

path = "jsons/*.json"
json_files = glob.glob(path)

file_names = [file.split("/")[-1] for file in json_files]
print(file_names)

['tema-12.json', 'tema-23.json', 'tema-16.json', 'tema-40.json', 'tema-84.json', 'tema-58.json', 'tema-83.json', 'tema-6.json', 'tema-73.json', 'tema-9.json', 'tema-1.json', 'tema-85.json', 'tema-36.json', 'tema-3.json', 'tema-76.json', 'tema-81.json', 'tema-33.json', 'tema-15.json', 'tema-54.json', 'tema-34.json', 'tema-17.json', 'tema-46.json', 'tema-74.json', 'tema-20.json', 'tema-65.json', 'tema-8.json', 'tema-18.json', 'tema-79.json', 'tema-52.json', 'tema-32.json', 'tema-48.json', 'tema-2.json', 'tema-75.json', 'tema-56.json', 'tema-55.json', 'tema-82.json', 'tema-14.json', 'tema-59.json', 'tema-78.json', 'tema-77.json', 'tema-31.json', 'tema-25.json', 'tema-42.json', 'tema-70.json', 'tema-60.json', 'tema-44.json', 'tema-11.json', 'tema-21.json', 'tema-38.json', 'tema-51.json', 'tema-50.json', 'tema-41.json', 'tema-68.json', 'tema-24.json', 'tema-53.json', 'tema-49.json', 'tema-45.json', 'tema-66.json', 'tema-26.json', 'tema-57.json', 'tema-80.json', 'tema-19.json', 'tema-67.json

In [5]:
lista_dfs = []

for chave in file_names:
    chave = chave.replace(".json", "")
    numero = chave.split('-')[1]

    # Read the JSON file into a dataframe
    df = pd.read_json(f'jsons/{chave}.json')

    # Display the dataframe
    print(df)
    df['tema'] = int(numero)
    lista_dfs.append(df)

df_total = pd.concat(lista_dfs)

df_total = df_total.drop(columns = ['texto_comentado', 'cometarios', 'titulo', 'link'], errors = 'ignore')


                                                  tema  \
0    Por que os casamentos se tornaram descartáveis...   
1    Por que os casamentos se tornaram descartáveis...   
2    Por que os casamentos se tornaram descartáveis...   
3    Por que os casamentos se tornaram descartáveis...   
4    Por que os casamentos se tornaram descartáveis...   
..                                                 ...   
115  Por que os casamentos se tornaram descartáveis...   
116  Por que os casamentos se tornaram descartáveis...   
117  Por que os casamentos se tornaram descartáveis...   
118  Por que os casamentos se tornaram descartáveis...   
119  Por que os casamentos se tornaram descartáveis...   

                                                 texto  \
0    Segundo o filósofo Nietzsche, ao pensar sobre ...   
1    De acordo com Zygmunt Bauman, filósofo e soció...   
2    Até que a morte nos separe ou que seja bom enq...   
3    No aplaudido filme “ História de um casamento”...   
4    Na coleç

In [6]:
df_total

,tema,texto,nota,id,competencias
0,12,"Segundo o filósofo Nietzsche, ao pensar sobre ...",160,18691,[{'competencia': 'Domínio da modalidade escrit...
1,12,"De acordo com Zygmunt Bauman, filósofo e soció...",160,18775,[{'competencia': 'Domínio da modalidade escrit...
2,12,Até que a morte nos separe ou que seja bom enq...,120,18781,[{'competencia': 'Domínio da modalidade escrit...
3,12,No aplaudido filme “ História de um casamento”...,120,18739,[{'competencia': 'Domínio da modalidade escrit...
4,12,"Na coleção de livros Percy Jackson, do Rick Ri...",160,18788,[{'competencia': 'Domínio da modalidade escrit...
...,...,...,...,...,...
115,30,A Atitude Blasé - termo proposto pelo sociólog...,160,16784,[{'competencia': 'Domínio da modalidade escrit...
116,30,"A série francesa, ‘’Lindinhas’’, foi acusada d...",160,16795,[{'competencia': 'Domínio da modalidade escrit...
117,30,É indubitável que a cultura de erotização infa...,200,16813,[{'competencia': 'Domínio da modalidade escrit...
118,30,O filme de origem francesa estreado pela NETFL...,160,16814,[{'competencia': 'Domínio da modalidade escrit...


### Extrai as notas que cada redação obteve em cada conceito


In [7]:
def get_competencias(coluna : pd.Series) -> str:
    """
    Essa função obtém o nome da competência contida na coluna

    Args:
        coluna: coluna que contem os dicionários da competência

    Essa função pega a coluna com os vários dicionarios, verifica se todos os dicionários 
    se referem à uma mesma competência e caso todos os dicionários se refiream à uma única competência, 
    retorna a string contendo o nome dessa competência
    """

    competencias = coluna.transform(lambda x:x['competencia']).unique()

    if(len(competencias) != 1):
        raise('Ou não há competencias, ou há mais de uma onde deveria ter apenas uma')
        
    label_competencia = competencias[0]

    return label_competencia

In [8]:
def process_all(df_entrada,coluna = 'nota'):
    """
    Extrai a partr da coluna competências, os nomes e notas das diferentes competências

    A coluna competências, é composta de dicionários contendo diferentes informações, entre elas, 
    os nomes das competências e as notas obtidas nas competências, essa função realiza preprocessamentos 
    extraindo as notas obitdas nas diferentes competências dos dicionários contidos nessas competências

    Args:
        df_entrada: dataframe a partir do qual vamos extrair as competências e as notas
        coluna: nome da chave nos dicionários que contem a nota da competência 
        (por algum motivo, no grande conjunto 1, essa chave é 'nota' como esperado, mas nos outros 
        dois conjuntos essa chave é 'motivo', possível bug no crawler)
    """
    
    ## 'expande' o conteudo contido na coluna competencias, em que cada registro é uma lista
    ## de dicionários, (cada dicionário associado a uma competência), 
    ## para varias colunas, cada uma associada a uma competência em que o registro 
    ## corresponde a um único dicionário (o dicionário conterá o nome da competencia a nota atribuida)
    ## e o motivo pelo qual a nota foi atribuida
    df_competencias = pd.DataFrame(df_entrada['competencias'].to_list())
    
    lista_competencias = df_competencias

    for item in lista_competencias:
      
        competencia = get_competencias(df_competencias[item])

        ## obtém a nota da competencia presente na coluna, a armazena numa coluna que informará a nota da competencia
        df_entrada[f'{competencia}_nota'] = df_competencias[item].transform(lambda x:x[coluna])
       
    return df_entrada

In [9]:
## marca os conjuntos
df_total['conjunto'] = 2
df_total['conjunto'] = np.where(df_total['tema'] <= 85, 1,df_total['conjunto'])
df_total['conjunto'] = np.where(df_total['tema'] >= 137, 3,df_total['conjunto']) 

## separa e refina cada conjunto separadamente
df_primeiro = df_total[df_total['conjunto'] == 1]
df_geral = process_all(df_primeiro)
df_geral = df_geral.drop(columns = ['nota','competencias','- Ruim_nota'],errors = 'ignore')


## o range dos targets 2 e 3 é de 0 a 10, com numeros quebrados, 
## multiplicamos por 100, esses targets e passamos todos os targets para valores inteiros,
## pois isso facilita o trabalho com kappa de cohen, mais a frente 

df_geral[ALL_TARGETS] = df_geral[ALL_TARGETS].astype(float)

In [10]:
df_geral

,tema,texto,id,conjunto,Domínio da modalidade escrita formal_nota,Compreender a proposta e aplicar conceitos das várias áreas de conhecimento para desenvolver o texto dissertativo-argumentativo em prosa_nota,"Selecionar, relacionar, organizar e interpretar informações em defesa de um ponto de vista_nota",Conhecimento dos mecanismos linguísticos necessários para a construção da argumentação_nota,Proposta de intervenção com respeito aos direitos humanos_nota
0,12,"Segundo o filósofo Nietzsche, ao pensar sobre ...",18691,1,160.0,120.0,120.0,200.0,200.0
1,12,"De acordo com Zygmunt Bauman, filósofo e soció...",18775,1,160.0,160.0,160.0,200.0,160.0
2,12,Até que a morte nos separe ou que seja bom enq...,18781,1,120.0,120.0,80.0,120.0,40.0
3,12,No aplaudido filme “ História de um casamento”...,18739,1,120.0,160.0,120.0,200.0,160.0
4,12,"Na coleção de livros Percy Jackson, do Rick Ri...",18788,1,160.0,120.0,120.0,160.0,0.0
...,...,...,...,...,...,...,...,...,...
115,30,A Atitude Blasé - termo proposto pelo sociólog...,16784,1,160.0,160.0,120.0,200.0,160.0
116,30,"A série francesa, ‘’Lindinhas’’, foi acusada d...",16795,1,120.0,120.0,120.0,160.0,160.0
117,30,É indubitável que a cultura de erotização infa...,16813,1,160.0,160.0,120.0,160.0,120.0
118,30,O filme de origem francesa estreado pela NETFL...,16814,1,160.0,120.0,120.0,160.0,120.0


###         Extrai features básicas e separa os datasets em treino e teste


In [11]:
def word_count(x):
    """
    Extrai a quantidade de palavras
    """
    lista = word_tokenize(x)
    return len(lista)

def word_count_unique(x):
    """
    Extrai a quantidade de palavras únicas
    """
    lista = set(word_tokenize(x))
    return len(lista)

def sentence_count(x):
    """
    Extrai a quantidade de sentenças
    """
    lista = sent_tokenize(x)
    return len(lista)


In [12]:
def generate_word_sentence_features(df : pd.DataFrame,column = 'texto') -> pd.DataFrame:
    """
    Extrai algumas features básicas dos textos

    Esse método extrai alguma features dos textos, como contagem de palavras, contagem de palavras únicas 
    e contagem de sentenças
    
    Args:
        df: datafame que contém as redações 
        column: nome da coluna que conterá os textos
    """

    df['word_count'] = df[column].apply(lambda x: word_count(x))
    df['unique_word_count'] = df[column].apply(lambda x: word_count_unique(x))
    df['sentence_count'] = df[column].apply(lambda x: sentence_count(x))
    
    return df


In [13]:
def separate_train_test(df: pd.DataFrame) -> tuple:
    """
    Separa os conjuntos de treino e de teste, estratificando pelo tema, 
    de modo a garantir que todos os temas possuam representação semelhante
    no treino e no teste

    Args: 
        df: dataframe que iremos separar em treino e teste
    """
    X_train, x_test = train_test_split(df, test_size=0.2,random_state=0, stratify = df['tema'])
    
    X_train = X_train.reset_index()
    x_test = x_test.reset_index()
    
    return X_train, x_test


In [25]:
df = generate_word_sentence_features(df_geral)
df_train,df_test = separate_train_test(df)

### Fit Vectorizer
    Método responsável por usar o conjunto de treino para treinar o vetorizador de textos 


In [26]:
lista = list(df_train['texto'])

dicio_tf_idf = {}

for count in TF_IDF_MAX_FEATURES:
    ## cria o vetorizador que utiliza TF-IDF
    tv = TfidfVectorizer(min_df = 10,max_features=count)

    ## treina o vetorizador com o conjunto fornecido para treino
    vectorizer_vez = tv.fit(lista)

    key = "TF_IDF_" + str(count)

    # prepara o dicionário para versionamento
    dicio_tf_idf[key] = vectorizer_vez


### Vectorize All
        Método responsavel por vetorizar os conjuntos de treino e teste


In [27]:

def vectorize_data(df: pd.DataFrame,model_dict: pickle) -> pd.DataFrame:
    """
    Usa um vetorizador TF-IDF pré-treinado para obter a representação vetorial para o conjunto de dados

    Args:
        df: dataset que contém as redações
        model: vetorizador TF-IDF pré-treinado
    """
    lista = list(df['texto'])

    dicio = {}

    for key,value in model_dict.items():

        model = value

        ## realiza efetivamente a vetorização, transformando em uma matriz esparsa
        X = model.transform(lista)

        # transforma a matriz esparsa em um dataframe organizado com as frequencias TF-IDF das palavras 
        df_vetorizado = pd.DataFrame(X.A, columns=model.get_feature_names_out())
        
        ## caso as redações contenham palavras que dão nome as features do datased
        ## (ex: texto, link, palavra) devemos removê-las

        colunas_a_remover = df.columns 

        df_vetorizado = df_vetorizado.drop(columns = colunas_a_remover, errors = 'ignore')

        df = pd.concat([df,df_vetorizado],axis = 1)
        df = df.drop(columns = ['texto'],errors = 'ignore')

        dicio[key] = df

 
    retorno = dicio
    
    return retorno, dicio


In [28]:
dicio_tf_idf

{'TF_IDF_32': TfidfVectorizer(max_features=32, min_df=10),
 'TF_IDF_64': TfidfVectorizer(max_features=64, min_df=10)}

In [29]:
df_train,df_train_list = vectorize_data(df_train,dicio_tf_idf)
df_test,df_test_list = vectorize_data(df_test,dicio_tf_idf)


## fit predict both ways
        Realiza o pipeline de treino e previsão tanto de forma geral, quanto de forma separada por tema


In [30]:

def fit_predict(df: pd.DataFrame) -> pd.DataFrame:
    """
    Realiza um pipeline de treino e teste dentro de um conjunto de textos

    Recebe um único conjunto com textos de treino e de teste concatenados, considerando esse conjunto 
    separa de volta os textos em treino e teste, treina o modelo com os textos de treino, 
    e usa esse modelo para realizar previsões no conjunto de teste 

    Args: 
        df: conjunto de redações
    """
    
    df_train = df[df['group'] == 'train'].drop(columns = ['group'])
    df_test = df[df['group'] == 'test'].drop(columns = ['group'])
    
    #id_train = df_train[ID_VARS]
    X_train = df_train.drop(columns = EXCLUDE_COLS,errors = 'ignore')
    y_train = df_train[TARGETS_1].astype(float)
    
    ## treina o modelo 
    xgb = XGBRegressor()
    fittado = xgb.fit(X_train, y_train)
    
    id_test = df_test[ID_VARS]
    X_test = df_test.drop(columns = EXCLUDE_COLS,errors = 'ignore')
    y_test = df_test[TARGETS_1].astype(float)
   
    PRED_COLS = [col + f'_pred' for col in TARGETS_1]
    
    preds = pd.DataFrame()
    preds[ID_VARS] = id_test
    preds[PRED_COLS] =  xgb.predict(X_test)
    preds[PRED_COLS] = preds[PRED_COLS].astype(int)

    return preds


In [31]:
df

,tema,texto,id,conjunto,Domínio da modalidade escrita formal_nota,Compreender a proposta e aplicar conceitos das várias áreas de conhecimento para desenvolver o texto dissertativo-argumentativo em prosa_nota,"Selecionar, relacionar, organizar e interpretar informações em defesa de um ponto de vista_nota",Conhecimento dos mecanismos linguísticos necessários para a construção da argumentação_nota,Proposta de intervenção com respeito aos direitos humanos_nota,word_count,unique_word_count,sentence_count
0,12,"Segundo o filósofo Nietzsche, ao pensar sobre ...",18691,1,160.0,120.0,120.0,200.0,200.0,380,216,8
1,12,"De acordo com Zygmunt Bauman, filósofo e soció...",18775,1,160.0,160.0,160.0,200.0,160.0,375,204,8
2,12,Até que a morte nos separe ou que seja bom enq...,18781,1,120.0,120.0,80.0,120.0,40.0,185,117,4
3,12,No aplaudido filme “ História de um casamento”...,18739,1,120.0,160.0,120.0,200.0,160.0,373,219,6
4,12,"Na coleção de livros Percy Jackson, do Rick Ri...",18788,1,160.0,120.0,120.0,160.0,0.0,586,287,25
...,...,...,...,...,...,...,...,...,...,...,...,...
115,30,A Atitude Blasé - termo proposto pelo sociólog...,16784,1,160.0,160.0,120.0,200.0,160.0,296,177,10
116,30,"A série francesa, ‘’Lindinhas’’, foi acusada d...",16795,1,120.0,120.0,120.0,160.0,160.0,523,294,12
117,30,É indubitável que a cultura de erotização infa...,16813,1,160.0,160.0,120.0,160.0,120.0,261,157,4
118,30,O filme de origem francesa estreado pela NETFL...,16814,1,160.0,120.0,120.0,160.0,120.0,134,97,5


In [32]:
df.columns

Index(['tema', 'texto', 'id', 'conjunto',
       'Domínio da modalidade escrita formal_nota',
       'Compreender a proposta e aplicar conceitos das várias áreas de conhecimento para desenvolver o texto dissertativo-argumentativo em prosa_nota',
       'Selecionar, relacionar, organizar e interpretar informações em defesa de um ponto de vista_nota',
       'Conhecimento dos mecanismos linguísticos necessários para a construção da argumentação_nota',
       'Proposta de intervenção com respeito aos direitos humanos_nota',
       'word_count', 'unique_word_count', 'sentence_count'],
      dtype='object')

In [33]:
df_train_list

{'TF_IDF_32':       index  tema     id  conjunto  Domínio da modalidade escrita formal_nota  \
 0        27    83  11659         1                                      160.0   
 1        57    39  15612         1                                      160.0   
 2        35    57  14071         1                                      160.0   
 3        23    26  17269         1                                      160.0   
 4       109    36  16059         1                                      200.0   
 ...     ...   ...    ...       ...                                        ...   
 6097     46    52  14327         1                                      160.0   
 6098    101    26  17203         1                                      160.0   
 6099     41    46  14979         1                                      120.0   
 6100     94    14  18522         1                                      160.0   
 6101      7    51  14392         1                                      120.0   
 
 

In [35]:
dict_pred_1 = {}
dict_pred_2 = {}

for key, value in df_train_list.items():
    df_train = df_train_list[key]
    df_test = df_test_list[key]

    print(key)

    df_train['group'] = 'train'
    df_test['group'] = 'test'
    df = pd.concat([df_train,df_test])
    df = df.drop(columns = 'texto',errors = 'ignore')
    pred1 = fit_predict(df)
    pred2 = df.groupby(['tema']).apply(lambda x: fit_predict(x)).reset_index(drop = True)
    dict_pred_1[key] = pred1
    dict_pred_2[key] = pred2   


TF_IDF_32


/tmp/ipykernel_279044/2960397382.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pred2 = df.groupby(['tema']).apply(lambda x: fit_predict(x)).reset_index(drop = True)


TF_IDF_64


/tmp/ipykernel_279044/2960397382.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pred2 = df.groupby(['tema']).apply(lambda x: fit_predict(x)).reset_index(drop = True)


In [36]:


def prepare_report_table(df_real: pd.DataFrame,df_pred : pd.DataFrame) -> pd.DataFrame:
    """
    Junta o conjunto de teste e as previsões feitas

    Juntamos o conjunto de treino com o conjunto de teste, fazemos um melt para
    transformar cada conceito em uma linha, e, com isso ficar mais fácil para realizar a avaliaçao 
    do desempenho preditivo em etapas futuras

    Args: 
        df_real: dataframe com o conjunto de teste
        df_pred: dataframe com as previsões realizadas tendo em mente
        o conjunto de teste
    """
    
    df_real = df_real[ID_VARS + TARGETS_1]
    df = pd.merge(df_real,df_pred, on =[ 'index','tema','conjunto'], suffixes = ['_real','_pred'])
    df = df.melt(id_vars = ['index','tema','conjunto'])
    df['valor'] = df['variable'].transform(lambda x:x.split('_')[-1])
    df['conceito'] = df['variable'].transform(lambda x:x.split('_')[0])
    df = df.drop(columns = 'variable')
    df = df.set_index(['index','tema','conjunto','conceito','valor']).unstack('valor').reset_index()
    df.columns = ['index','tema','conjunto','conceito','nota','previsao']

    return df



In [37]:
dicio = {}

for key,value in df_test_list.items():

    df_pred_geral = dict_pred_1[key]
    df_pred_especifica = dict_pred_2[key]
    df_real = df_test_list[key]
    
    report_geral = prepare_report_table(df_real,df_pred_geral)
    report_especifica = prepare_report_table(df_real,df_pred_especifica)


    res = report_geral.groupby(['conceito']).apply(lambda x: cohen_kappa_score(x['nota'],x['previsao']))
    score_geral = pd.DataFrame(data = res).reset_index()
    score_geral.columns = ['conceito','score_geral']

    res = report_especifica.groupby(['conceito']).apply(lambda x: cohen_kappa_score(x['nota'],x['previsao']))
    score_especifica = pd.DataFrame(data = res).reset_index()
    score_especifica.columns = ['conceito','score_especifica']

    df_score = pd.merge(score_geral,score_especifica, on = ['conceito'])

    dicio[key] = df_score

/tmp/ipykernel_279044/3927120229.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = report_geral.groupby(['conceito']).apply(lambda x: cohen_kappa_score(x['nota'],x['previsao']))
/tmp/ipykernel_279044/3927120229.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = report_especifica.groupby(['conceito']).apply(lambda x: cohen_kappa_score(x['nota'],x['previsao']))
/tmp/ipykernel_279044/392712022

In [40]:
dicio

{'TF_IDF_32':                                             conceito  score_geral  \
 0  Compreender a proposta e aplicar conceitos das...    -0.000480   
 1  Conhecimento dos mecanismos linguísticos neces...    -0.002404   
 2               Domínio da modalidade escrita formal     0.003298   
 3  Proposta de intervenção com respeito aos direi...     0.000549   
 4  Selecionar, relacionar, organizar e interpreta...     0.003629   
 
    score_especifica  
 0         -0.001339  
 1         -0.002801  
 2         -0.002665  
 3         -0.000683  
 4          0.000302  ,
 'TF_IDF_64':                                             conceito  score_geral  \
 0  Compreender a proposta e aplicar conceitos das...    -0.000410   
 1  Conhecimento dos mecanismos linguísticos neces...    -0.000677   
 2               Domínio da modalidade escrita formal     0.001123   
 3  Proposta de intervenção com respeito aos direi...     0.001791   
 4  Selecionar, relacionar, organizar e interpreta...     0.000